In [2]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys
import re
from glob import glob
import csv

import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError

from humanfriendly import prompts

# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [3]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [4]:
def fileSearch(path = None, search = None):
        '''search for a file name string in a given path'''
        if not search:
            search = ''
        logger = logging.getLogger(__name__)
        logger.debug('searching path: {0} for \"{1}\"'.format(path, search))
        try:
            searchPath = os.path.expanduser(path)
        except AttributeError as e:
            logger.error('Error: no path specified')
            searchPath = ''
        try:
            allFiles = os.listdir(searchPath)
        except OSError as e:
            logger.error(e)
            return(None)
        
        # list comprehension search with regex
        #http://www.cademuir.eu/blog/2011/10/20/python-searching-for-a-string-within-a-list-list-comprehension/
        regex = re.compile('.*('+search+').*')
        return([m.group(0) for l in allFiles for m in [regex.search(l)] if m])

In [5]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.info('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [6]:
def getTeamDrive(myDrive):
    '''
    menu driven interaction for choosing a writeable team drive
    '''
    #available and writeable team drives
    logger = logging.getLogger(__name__)
    try:
        teamdrives_all = myDrive.listTeamDrives()
    except Exception as e:
        logger.error(e)
    
    if len(teamdrives_all) < 1:
        logger.warn('no writeable team drives found for this user')
        return(None)
    
    teamdrives_writeable = {}
    for drive in teamdrives_all:
        if drive['capabilities']['canAddChildren']:
            teamdrives_writeable[drive['name']] = drive
    
    teamdrive = None
    
    try:
        teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None)
        teamdrive=teamdrives_writeable[teamdrive_name]
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
    
    return(teamdrive)

In [7]:
def getPortfolioFolder(myDrive):
    '''
    menu driven interaction to locate and choose portfolio folder on team drive
    
    returns:
        folder (dictionary) - google drive object properties dictionary
    '''
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive: {}; {}'.format(teamdriveName, teamdriveID))
    
    foldersearch = ''
    attempt = 5
    while (len(foldersearch) <= 0) and (attempt > 0):
        attempt -= 1
        foldersearch = prompts.prompt_for_input(question='Please enter a portion of the portfolio folder name (case insensitive): ',
                                                default=None, strip=True)
        if len(foldersearch) <= 0:
            print 'Nothing entered. {} attempts remain.'.format(attempt)
            
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        if len(searchresult['files']) <= 0:
            print 'No folders contained "{}"'.format(foldersearch)
            print 'Please try entering only part of the folder name'
            foldersearch = ''
            
    
    
    if foldersearch <= 0:
        logger.warn('user entered 0 length string for folder search')
        return(None)
    
    
    foldernames = []
    for each in searchresult['files']:
        foldernames.append(each['name'])
    
    try:
        foldername = prompts.prompt_for_choice(choices=foldernames)
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
        return(None)
    
    return(searchresult['files'][foldernames.index(foldername)])
    

In [8]:
def getPathfromList(list_path=['~/'], message='Choose from the paths below', default=None):
    '''
    menu driven interaction to select path 
    accepts:
        list_path (list) - list of paths to display
        messager (string) - message to display
        default (string) - default choice (must be one of the items in the list otherwise defaults to None)
    
    returns:
        searchPath (string) - valid and existing path
    '''
    logger = logging.getLogger(__name__)
    logger.debug('path list: {}'.format(list_path))
    searchPath = None
    list_path = sorted(list_path)
    list_path.append('OTHER')
    
    if default not in list_path:
        logging.warn('default ({}) not in list; setting default = None'.format(default))
        default = None
    
    if message:
        print message
    searchPath = prompts.prompt_for_choice(choices=list_path, default=default)
    logger.debug('searchPath = {}'.format(searchPath))
    
    if not searchPath is 'OTHER':
        searchPath = os.path.expanduser(searchPath)
    else:
        attempt = 5
        searchPath = None
        while not searchPath and attempt > 0:
            logger.debug('attempts remaining: {}'.format(attempt))
            attempt -= 1
            searchPath = prompts.prompt_for_input('Please type the complete path to use: ')
            logger.debug('searchPath = {}'.format(searchPath))         
    
    if len(searchPath) < 1:
        searchPath = None
    return (searchPath) 

In [9]:
def getFiles(path='~/', pattern='.*', ignorecase=True):
    '''
    search path for files matching pattern (regular expression)
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case'''
    logger = logging.getLogger(__name__)
    
    path = os.path.expanduser(path+'/*')
    logger.debug('Path: {}, glob pattern: {}, ignorecase: {}'.format(path, pattern, ignorecase))
    
    flags = re.IGNORECASE
    files = []
        
    if ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f, flags=flags)]:
            files.append(eachfile)
    
    if not ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f)]:
            files.append(eachfile)        
    
    return(files)

In [10]:
def chooseFile(path='~/', pattern='.*', ignorecase=True, message='Please choose a file from the list'):
    '''
    menu interaction for choose a file in the specified path from the file glob created using a regex pattern
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case
        message(string) - message to show
    returns:
        selection (string) - selected file'''
    
    logger = logging.getLogger(__name__)
    filelist = getFiles(path=path, pattern=pattern, ignorecase=ignorecase)
    logger.debug('filelist: {}'.format(filelist))
    print message
    selection = prompts.prompt_for_choice(choices=filelist)
    return(selection)

In [11]:
def fileToList(inputfile):
    logger = logging.getLogger(__name__)
    logger.debug('inputfile = {}'.format(inputfile))
    outputlist = []
    
    try:
        with open(inputfile, 'r') as fhandle:
            logger.debug('reading file: {}'.format(inputfile))
            for line in fhandle:
                outputlist.append(line.strip('\n'))
    except IOError as e:
        logger.debug(e)
    return(outputlist)

In [12]:
def mapHeaders(file_csv, expected_headers=[]):
    '''map an expected list of header values to their position in a csv
    accepts:
        file_csv (csv.reader object) 
        expected_headers (list) - list of headers to search for, ignoring all others
    '''
    logger = logging.getLogger(__name__)
    logger.debug('mapping headers')
    
    missing_headers = []
    headerMap = {} 
    
    try:
        csvHeader = file_csv[0]
    except IndexError as e:
        logging.error('csv empty: {}'.format(e))
        return(None)
    
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
    
    headerMap['missingheaders'] = missing_headers
    
    headerMap['headers'] = {}

    if len(missing_headers) > 0:
        logging.warn('missing headers: {}'.format(missing_headers))
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            headerMap['headers'][value] = index
    logger.debug('successfully maped headers')
    return(headerMap)

In [41]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))

studentexport_list = ['~/Downloads', '~/Desktop']
studentexport = None
studentexport_path = None


downloadlink = 'https://foo.foof.foo'


updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

teamdriveName = '**UNKNOWN**'

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = './gradefolders.txt'


    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logger.debug('current configuration:')
    logger.debug('\n{}'.format(config_dict))

# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
if not prompts.prompt_for_confirmation(question='Would you like to proceed?', default='y'):
    print 'Exiting'
    exit(0)
    
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if not teamdriveID:
    logger.warn('Team Drive ID missing')
    try:
        teamdrive = getTeamDrive(myDrive)
    except GDriveError as e:
        logging.error('Error accessing team drives: e')
        exit(0)
        
    if not teamdrive:
        logger.error('no team drives found')
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print('No Team Drives located; cannot proceed.')
        print('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        logger.error('exiting')
        exit(0)
        
    try:
        teamdriveID = teamdrive['id']
        teamdriveName = teamdrive['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid team drive information found: {}'.format(e))
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print 'No Team Drive was found. Do you have write access to a Team Drive with this account?'
        print 'You are authenticated as: {}'.format(myDrive.userinfo['emailAddress'])
        logger.error('exiting')
        exit(0)
        
    print 'Using Team Drive: {}'.format(teamdriveName)    
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdriveName', teamdriveName)
    updateConfig = True
    
if not folderID:
    logger.warn('Folder ID missing')
    try:
        folder = getPortfolioFolder(myDrive)
    except GDriveError as e:
        logger.error('Could not search for folder: {}'.format(e))
        print ('There was a problem searching Team Drive for a folder')
        print ('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        print ('If this is not correct do...')
    try:
        folderID = folder['id']
        folderName = folder['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid folder information found {}'.format(e))
        logger.error('exiting')
        exit(0)
    
    print 'Using Folder: {} on Team Drive: {}'.format(folderName, teamdriveName)
    myConfig.set('Main', 'foldername', folderName)
    myConfig.set('Main', 'folderid', folderID)
    updateConfig=True

if not os.path.exists(gradefolders):
    logger.error('failed to locate {}'.format(gradefolders))
    logger.error('exiting')
    print 'gradefolders.txt file is missing. Please reinstall the application.'
    print 'download link: {}'.format(downloadlink)
    exit(0)

# get the gradefolder file
gradefolder_list = fileToList(gradefolders)

# check validity of gradefolder
if len(gradefolder_list) < 1:
    print 'grade folder file is corrupt. Please reinstall the application'
    exit(0)


if updateConfig:
    configuration.create_config(cfgfile, myConfig)


# # get and validate path for student_export folder
attempt = 5
while not studentexport and (attempt > 0):
    logger.debug('remaining attempts to locate student export: {}'.format(attempt))
    attempt -= 1
    if not studentexport:
        studentexport_path = getPathfromList(studentexport_list, 
                                             message='Please choose which folder contains the Student Export File.',
                                             default=studentexport_list[0])
    
    # move this pattern into the configuration file? 
    studentexport = chooseFile(path=studentexport_path, pattern='.*student.*export.*')
    logger.debug('student export file chosen: {}'.format(studentexport))
    if not os.access(studentexport, os.R_OK):
        print 'Could not read file: {}. Please choose another file'.format(studentexport)
        logger.error('file is unreadable: {}'.format(studentexport))
        studentexport = None

if not studentexport:
    logger.error('no student export file; exiting')
    print 'Cannot proceed without a student export file. Exiting'
    exit(0)
else:
    myConfig.set('Main', 'studentexport', studentexport)
    studentexport_list = fileToList(studentexport)
        
logger.debug('Student export file: {}'.format(studentexport))

studentexport_csv = []
try: 
    with open(studentexport, 'rU') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            studentexport_csv.append(row)
except (OSError, IOError) as e:
    logging.critical('error reading file: {}\n{}'.format(studentexport,e))
    print 'Could not read file: {}; exiting.'.format(studentexport)
    exit(0)

# parse and check validity of student export file
expected_headers = ['ClassOf', 'Student_Number', 'LastFirst']
headerMap = mapHeaders(file_csv=studentexport_csv, expected_headers=expected_headers)

if not headerMap or (len(headerMap['headers']) != len(expected_headers)):
    logging.error('cannot continue without valid header map; exiting')
    print('file: {} appears to not contain the expected header row (in any order): {}'.format(studentexport, expected_headers))
    if headerMap['missingheaders']:
        print('your file appears to be missing the field(s): {}'.format(headerMap['missingheaders']))
    print('please try to download a new student.export file before trying again; exiting')
#     exit(0)

logger.debug('asking to proceed with current configuration')
print '\nCurent Configuration:'
for section in myConfig.sections():
    print '[{}]'.format(section)
    for option in myConfig.options(section):
        print '{} = {}'.format(option, myConfig.get(section, option))
if not prompts.prompt_for_confirmation('Continue with the configuration above?'):
    logger.debug('user chose to exit')
    exit(0)

2018-09-17 13:11:10,981: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-09-17 13:11:10,993: [DEBUG: configuration.get_config] reading configuration file at: /Users/aciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-09-17 13:11:10,999: [DEBUG: __main__.<module>] current configuration:
2018-09-17 13:11:11,000: [DEBUG: __main__.<module>] 
{'Main': {'credentials': '/Users/aciuffo/.config/portfolio_creator/credentials/', 'folderid': '1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', 'teamdriveid': '0ACLfU8KeD_BHUk9PVA', 'teamdrivename': 'IT Blabla', 'foldername': 'Port Wine'}}
2018-09-17 13:11:11,003: [INFO: root.<module>] checking google credentials




You *may* be directed to a google web page that asks you to authorize this applicaiton.
Please choose an ASH account and authorize this applicaiton.
When you are done, please close the newly created tab in your web browser and return to this window.
2018-09-17 13:11:11,006: [DEBUG: humanfriendly.prompts.prompt_for_confirmation]

2018-09-17 13:11:13,704: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/teamdrives?fields=teamDrives&alt=json
2018-09-17 13:11:14,116: [DEBUG: __main__.fileToList] inputfile = ./gradefolders.txt
2018-09-17 13:11:14,119: [DEBUG: __main__.fileToList] reading file: ./gradefolders.txt
2018-09-17 13:11:14,123: [DEBUG: __main__.<module>] remaining attempts to locate student export: 5
2018-09-17 13:11:14,126: [DEBUG: __main__.getPathfromList] path list: ['~/Downloads', '~/Desktop']
Please choose which folder contains the Student Export File.
2018-09-17 13:11:14,130: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are '~/Desktop', '~/Downloads' and 'OTHER') ..

  1. ~/Desktop
  2. ~/Downloads (default choice)
  3. OTHER
 
 Enter your choice as a number or unique substring (Control-C aborts): 2
2018-09-17 13:11:16,474: [DEBUG: humanfriendly.prompts.prompt_for_choice] Option ('~/Downloads')


 Continue with the configuration above? [y/n] 
2018-09-17 13:11:25,152: [DEBUG: humanfriendly.prompts.prompt_for_confirmation] Got empty reply (there's no default choice), retrying (1/10) ..



 Error: Please enter 'yes' or 'no' (there's no default choice).



 Continue with the configuration above? [y/n] y
2018-09-17 13:11:26,368: [DEBUG: humanfriendly.prompts.prompt_for_confirmation] Confirmation granted by reply ('y').


In [56]:
def createFolders(myDrive, parentFolder, student_csv, header_map):
    '''
    create folders on google drive if they do not exist
    
    accepts:
        myDrive(googleDrive object)
        folderid (string) - folder id string
        folders (list) - list of folder names
    
    returns:
        folderinfo (dict) - {folder name: folder url, or None for failures}
    '''
    logger = logging.getLogger(__name__)
    logger.info('creating folders at gdrive path: {}'.format(folderID))
    
    myDrive.getprops
    
    folderinfo = {}
    
    # check that folder exists and is a folder
    try:
        logger.info('getting folder properties')
        props = myDrive.getprops(folderID, 'capabilities, mimeType, name')
    except GDriveError as e:
        logger.error('failed to get properties for {}; {}'.format(folderID, e))
        logger.error('please reconfigure the folder option. cannot continue; exiting')
        exit(0)

    if not props['mimeType'] == myDrive.mimeTypes['folder']:
        logger.error('{} is not a folder; type: {}'.format(folderID, props['mimeType']))
    else:
        logger.info('folder: {} is indeed a folder'.format(props['name']))
        
    if not props['capabilities']['canAddChildren']:
        logger.error('folder: {} is not writable'.format(props['name']))
    else:
        logger.info('folder: {} is writeable'.format(props['name']))
        
    
    logging.info('searching for exisiting folders and creating missing folders')
    for folder in folders:
        pass
    
    
    
    return(folderinfo)
    
    
    
    

In [68]:
x = set([1, 2, 3])
x.add(5)
for each in x:
    print each

1
2
3
5


In [69]:
folderID

'1-D7UNBes_skfkQ6oBettiBICiBcZmbvn'

In [90]:
classOf_set = set(['3099'])
name_set = set([])
classOf_string = 'Class Of-'
# gather all unique student names and classOf rows
logger.debug('creating "Class Of" folders as needed')
for student in studentexport_csv[1:]:
    classOf_set.add(student[headerMap['headers']['ClassOf']])
    name_set.add(student[headerMap['headers']['LastFirst']]+' - '+student[headerMap['headers']['Student_Number']])

# check for existence of classOf folders and create if they are missing
for eachClass in classOf_set:
    # assume folder needs to be created
    createfolder=True
    try:
        class_result = myDrive.search(name=classOf_string+eachClass, parents=folderID, teamdrive=teamdriveID)
    except GDriveError as e:
        logger.error(GDriveError)
        
    for eachFile in class_result['files']:
        # if a folder type with appropriate name is found, cancel creation
        if eachFile['mimeType'] == myDrive.mimeTypes['folder']:
            logger.info('Folder exists: {}'.format(eachClass))
            createfolder=False

    # create the folder
    if createfolder:
        logger.info('Create folder: {}'.format(eachClass))
    

2018-09-17 14:21:22,045: [DEBUG: __main__.<module>] creating "Class Of" folders as needed
2018-09-17 14:21:22,049: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-3099" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime))
2018-09-17 14:21:22,061: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-3099%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-17 14:21:22,459: [INFO: __main__.<module>] Folder exists: 3099
2018-09-17 14:21:22,462: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2029" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime))
2018-09-17 14:21:22,476: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?order

In [80]:
myDrive.mimeTypes
# myDrive.search(name='Class Of-3099', parents=folderID, teamdrive=teamdriveID)

{'audio': 'application/vnd.google-apps.audio',
 'docs': 'application/vnd.google-apps.document',
 'drawing': 'application/vnd.google-apps.drawing',
 'file': 'application/vnd.google-apps.file',
 'folder': 'application/vnd.google-apps.folder',
 'forms': 'application/vnd.google-apps.form',
 'mymaps': 'application/vnd.google-apps.map',
 'photos': 'application/vnd.google-apps.photo',
 'scripts': 'application/vnd.google-apps.script',
 'sheets': 'application/vnd.google-apps.spreadsheet',
 'sites': 'application/vnd.google-apps.sites',
 'slides': 'application/vnd.google-apps.presentation',
 'video': 'application/vnd.google-apps.video'}

In [50]:

headerMap

{'headers': {'ClassOf': 2, 'LastFirst': 0, 'Student_Number': 1},
 'missingheaders': []}

In [45]:
studentexport_csv

[['LastFirst', 'Student_Number', 'ClassOf'],
 ['Al Abdulkareem, Abdulaziz', '503593', '2029'],
 ['Araneda Martinez, Sebastian', '504864', '2029'],
 ['Arias, Angela Therese', '503509', '2029'],
 ['Bartolone, Lana', '503845', '2029'],
 ['Bates, Gage', '504450', '2029'],
 ['Beltran Cui, Nina', '503835', '2029'],
 ['Bieler, Alwin', '503855', '2029'],
 ['Bott, Elliott', '504114', '2029'],
 ['Callender, Anna Margaret', '504755', '2029'],
 ['Chu, Euan', '504144', '2029'],
 ['Costello, Ryan', '503993', '2029'],
 ['Crane, Jillian', '504661', '2029'],
 ['Davis, Franklin', '503994', '2029'],
 ['Dolinskyy, Oleksandr', '504136', '2029'],
 ['Escalona Bolea, Candela', '504773', '2029'],
 ['Escalona Bolea, Lucas', '504774', '2029'],
 ['Escorial Gorriti, Mario', '504437', '2029'],
 ['Fischer, Elvira', '504601', '2029'],
 ['Fridenson, Itay', '503866', '2029'],
 ['Goddard, Maddison', '504108', '2029'],
 ['Graf, Peyton', '504951', '2029'],
 ['Gutwirth, Yali', '505088', '2029'],
 ['Harkonen, Amos', '504953

In [21]:
import random

In [22]:
createFolders(myDrive, '1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', [random.choice(['a','b','c','d','e','f','g'])+str(random.randint(1, 1000)) for _ in range(10)])

2018-09-17 12:55:59,827: [INFO: __main__.createFolders] creating folders at gdrive path: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-17 12:55:59,830: [INFO: __main__.createFolders] getting folder properties
2018-09-17 12:55:59,832: [DEBUG: gdrive.gdrive.getprops] files().get(fileId=1-D7UNBes_skfkQ6oBettiBICiBcZmbvn, fields=capabilities,mimeType,name)
2018-09-17 12:55:59,842: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files/1-D7UNBes_skfkQ6oBettiBICiBcZmbvn?fields=capabilities%2CmimeType%2Cname&alt=json&supportsTeamDrives=true
2018-09-17 12:56:00,186: [INFO: __main__.createFolders] folder: Port Wine is indeed a folder
2018-09-17 12:56:00,188: [INFO: __main__.createFolders] folder: Port Wine is writeable
2018-09-17 12:56:00,191: [INFO: root.createFolders] searching for exisiting folders and creating missing folders


{}

In [38]:
myDrive.search(name='Class Of-3099', parents='1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', teamdrive='0ACLfU8KeD_BHUk9PVA', quiet=False)

2018-09-17 13:09:08,401: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-3099" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime))
q=name = "Class Of-3099" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime)
2018-09-17 13:09:08,415: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-3099%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json


{u'files': [{u'id': u'0B9WTleJ1MzaYS1J1d3VhNGVjVnM',
   u'kind': u'drive#file',
   u'mimeType': u'application/vnd.google-apps.folder',
   u'name': u'Class Of-3099',
   u'teamDriveId': u'0ACLfU8KeD_BHUk9PVA'}],
 u'incompleteSearch': False,
 u'kind': u'drive#fileList'}

In [42]:
headerMap

{'headers': {'ClassOf': 2, 'LastFirst': 0, 'Student_Number': 1},
 'missingheaders': []}

In [17]:
help(myDrive.search)

Help on method search in module gdrive.gdrive:

search(self, name=None, trashed=None, mimeType=False, fuzzy=False, modifiedTime=None, dopperator='>', parents=None, orderBy='createdTime', teamdrive=None, quiet=True) method of gdrive.gdrive.googledrive instance
    search for an item by name and other properties in google drive using drive.files.list
    
    args:
        name (string): item name in google drive - required
        trashed (bool): item is not in trash - default None (not used)
        mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
        fuzzy = (bool): substring search of names in drive
        date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
        dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
        parents = (string): google drive file id string
        orderBy = (comma separated string): order results assending by keys below - default createdTime:
                    

In [ ]:
import random


In [ ]:
mylist = [random.choice(['a','b','c','d','e','f','g'])+str(random.randint(1, 1000)) for _ in range(10)]
print mylist
# print random.choice(['a','b','c','d','e','f','g'])+str(random.randint(1, 1000))

In [ ]:
help(random.randint)

In [ ]:
myDrive.getprops('1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', 'capabilities(canAddChildren), mimeType, name')

In [ ]:
myDrive.fields

In [ ]:
createFolders(myDrive=myDrive, folderID=folderID, folders=['a', 'b', 'cow'])

In [ ]:
help(myDrive.getprops)

In [ ]:
myDrive.mimeTypes